In [ ]:
#! pip install -q kaggle
from google.colab import files

files.upload()  # Choose the kaggle.json file that you downloaded

Saving kaggle.json to kaggle (2).json


{'kaggle (2).json': b'{"username":"yitzhakgrinvald","key":"e174162200b831fe822116cd99c868d4"}'}

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import pandas as pd


Make directory named kaggle and copy kaggle.json file ther

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

mkdir: cannot create directory ‘/root/.kaggle’: File exists


Change the permissions of the file:

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d alxmamaev/flowers-recognition

flowers-recognition.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip -qq flowers-recognition.zip

# הכנת התיקיות לטובת פיצול לtrain, val, test

In [ ]:
import os
import glob

# Define the source and target directories
source_parent_directory = "flowers"
target_parent_directory = "flowers_new"

# Ensure the target directory exists
os.makedirs(target_parent_directory, exist_ok=True)

# Iterate over the flower categories
flower_categories = ['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']

for category in flower_categories:
    source_directory = os.path.join(source_parent_directory, category)
    target_directory = os.path.join(target_parent_directory, category)

    # Ensure the target directory for the current category exists
    os.makedirs(target_directory, exist_ok=True)

    # Initialize a counter for the new numeric names
    new_name_counter = 0

    # Iterate over image files in the source directory (assuming they are in .jpg format)
    for image_file in glob.glob(os.path.join(source_directory, "*.jpg")):
        # Generate the new filename with the desired format (e.g., "0.jpg", "1.jpg", ...)
        new_filename = f"{new_name_counter}.jpg"

        # Create the new file's full path in the target directory
        target_filepath = os.path.join(target_directory, new_filename)

        # Rename and move the file
        os.rename(image_file, target_filepath)

        # Increment the counter for the next file
        new_name_counter += 1



פיצול הדאטה ל train, val, test

In [ ]:
import shutil
import pathlib

original_dir = pathlib.Path("flowers_new")
new_base_dir = pathlib.Path("flowers_new_split")

def make_subset(subset_name, start_index, end_index):
    for category in ("daisy", "dandelion", "rose","sunflower",  "tulip"):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir, exist_ok=True)  # Create directories if they don't exist
        fnames = [f"{i}.jpg" for i in range(start_index, end_index)]
        for fname in fnames:
            # Adjust the source and destination paths to match your directory structure
            src_path = original_dir / category / fname
            dst_path = dir / fname
            shutil.copyfile(src=src_path, dst=dst_path)

make_subset("train", start_index=0, end_index=590)
make_subset("validation", start_index=590, end_index=660)
make_subset("test", start_index=660, end_index=730)


In [ ]:

# Define the number of classes (5 in your case)
num_classes = 5

# Input layer
inputs = keras.Input(shape=(180, 180, 3))

# Data preprocessing layer (you can keep this as it is)
x = layers.Rescaling(1./255)(inputs)

# Convolutional layers with max pooling
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)

# Flatten the output of the convolutional layers
x = layers.Flatten()(x)

# Output layer for multi-class classification (5 output units)
outputs = layers.Dense(num_classes, activation="softmax")(x)

# Define the model
model = keras.Model(inputs=inputs, outputs=outputs)


# New Section

In [ ]:
model.compile(loss="categorical_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [ ]:
from tensorflow.keras.utils import image_dataset_from_directory

train_dataset = image_dataset_from_directory(
    new_base_dir / "train",
    image_size=(180, 180),
    batch_size=32)
validation_dataset = image_dataset_from_directory(
    new_base_dir / "validation",
    image_size=(180, 180),
    batch_size=32)
test_dataset = image_dataset_from_directory(
    new_base_dir / "test",
    image_size=(180, 180),
    batch_size=32)


Found 2950 files belonging to 5 classes.
Found 350 files belonging to 5 classes.
Found 350 files belonging to 5 classes.


In [ ]:
import tensorflow as tf
from tensorflow import keras

# Assuming you have your model defined as shown in your initial code

# Define the number of classes
num_classes = 5

# Update the model's output layer for multi-class classification
model.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",  # Use categorical crossentropy for multi-class
    metrics=["accuracy"]
)

# Define callbacks (no need to change this part)
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="Flower_identification.keras",
        save_best_only=True,
        monitor="val_loss"
    )
]

# Assuming you have your train_dataset and validation_dataset ready
history = model.fit(
    train_dataset,
    epochs=10,
    validation_data=validation_dataset,
    callbacks=callbacks
)


Epoch 1/10
93/93 [==============================] - 10s 87ms/step - loss: 1.5161 - accuracy: 0.3468 - val_loss: 2.4224 - val_accuracy: 0.2543
Epoch 2/10
93/93 [==============================] - 6s 58ms/step - loss: 1.2113 - accuracy: 0.5088 - val_loss: 1.2110 - val_accuracy: 0.5086
Epoch 3/10
93/93 [==============================] - 8s 77ms/step - loss: 1.0707 - accuracy: 0.5664 - val_loss: 1.0894 - val_accuracy: 0.6057
Epoch 4/10
93/93 [==============================] - 6s 59ms/step - loss: 0.9776 - accuracy: 0.6159 - val_loss: 0.9091 - val_accuracy: 0.6629
Epoch 5/10
93/93 [==============================] - 7s 76ms/step - loss: 0.9035 - accuracy: 0.6576 - val_loss: 3.2415 - val_accuracy: 0.3371
Epoch 6/10
93/93 [==============================] - 6s 58ms/step - loss: 0.8496 - accuracy: 0.6902 - val_loss: 1.0626 - val_accuracy: 0.6314
Epoch 7/10
93/93 [==============================] - 6s 59ms/step - loss: 0.7722 - accuracy: 0.7142 - val_loss: 0.9671 - val_accuracy: 0.6257
Epoch 8/10
9

In [ ]:
import matplotlib.pyplot as plt
accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs, accuracy, "bo", label="Training accuracy")
plt.plot(epochs, val_accuracy, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()

**Evaluating the model on the test set**

In [ ]:
test_model = keras.models.load_model("Flower_identification.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
basic_model= "Test accuracy: "+str(test_acc)
print(f"Test accuracy: {test_acc:.3f}")

# Using data augmentation

### Define a data augmentation stage to add to an image model

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ]
)

**Displaying some randomly augmented training images**

In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in train_dataset.take(1):
    for i in range(9):
        augmented_images = data_augmentation(images)
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(augmented_images[0].numpy().astype("uint8"))
        plt.axis("off")

**Defining a new convnet that includes image augmentation and dropout**

In [ ]:
num_classes = 5

inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(x)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

model.compile(loss="sparse_categorical_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])


callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="Flower_identification_with_augmentation.keras",
        save_best_only=True,
        monitor="val_loss")
]
history = model.fit(
    train_dataset,
    epochs=14,
    validation_data=validation_dataset,
    callbacks=callbacks)


**Training the regularized networks**

In [ ]:
import matplotlib.pyplot as plt
accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs, accuracy, "bo", label="Training accuracy")
plt.plot(epochs, val_accuracy, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()

**evaluating the the new model**

In [ ]:
test_model = keras.models.load_model(
    "Flower_identification_with_augmentation.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
augm_model= "Test accuracy: "+str(test_acc)
print(f"Test accuracy: {test_acc:.3f}")


# Leveraging a pretrained model

**Feature extraction with a pretrained model**

Instantiating the VGG16 convolutional base

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import RMSprop


In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(180, 180, 3))
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(5, activation='softmax'))  # Adjust the output layer units according to your number of flower classes (5 in this case)
model.compile(optimizer=RMSprop(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])


history = model.fit(train_dataset, epochs=14, validation_data=validation_dataset, callbacks=callbacks)


58889256/58889256 [==============================] - 0s 0us/step
Epoch 1/14
135/135 [==============================] - 69s 424ms/step - loss: 1.7466 - accuracy: 0.3113 - val_loss: 1.2548 - val_accuracy: 0.4719
Epoch 2/14
135/135 [==============================] - 50s 371ms/step - loss: 1.0459 - accuracy: 0.5979 - val_loss: 0.7618 - val_accuracy: 0.7142
Epoch 3/14
135/135 [==============================] - 50s 368ms/step - loss: 0.7696 - accuracy: 0.7148 - val_loss: 1.6897 - val_accuracy: 0.5133
Epoch 4/14
135/135 [==============================] - 50s 370ms/step - loss: 0.6381 - accuracy: 0.7855 - val_loss: 0.4226 - val_accuracy: 0.8522
Epoch 5/14
135/135 [==============================] - 50s 370ms/step - loss: 0.5866 - accuracy: 0.7975 - val_loss: 0.2870 - val_accuracy: 0.8965
Epoch 6/14
135/135 [==============================] - 50s 367ms/step - loss: 0.4922 - accuracy: 0.8379 - val_loss: 0.3157 - val_accuracy: 0.8918
Epoch 7/14
135/135 [==============================] - 50s 370ms/s

In [ ]:
# Assuming you have your test_dataset ready
test_accuracy = model.evaluate(test_dataset)
VGG16_model= "Test accuracy: "+str(test_acc)
print("Test accuracy:", test_accuracy[1])  # Index 1 corresponds to accuracy


improve the model by add dropout and regularization

In [ ]:
from tensorflow.keras.regularizers import l2

# Define the number of classes (5 in your case)
num_classes = 5

# Input layer
inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)
# Data preprocessing layer (you can keep this as it is)
x = layers.Rescaling(1./255)(inputs)

# Convolutional layers with max pooling
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)

# Add Dropout Layers
x = layers.Dropout(0.5)(x)

# Flatten the output of the convolutional layers
x = layers.Flatten()(x)

# Add L2 Regularization to Dense Layers
x = layers.Dense(256, activation='relu', kernel_regularizer=l2(0.01))(x)
x = layers.Dense(128, activation='relu', kernel_regularizer=l2(0.01))(x)

# Output layer for multi-class classification (5 output units)
outputs = layers.Dense(num_classes, activation="softmax")(x)

# Define the model
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])


callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="Flower_identification_with_augmentation_dropout.keras",
        save_best_only=True,
        monitor="val_loss")
]
history = model.fit(
    train_dataset,
    epochs=50,
    validation_data=validation_dataset,
    callbacks=callbacks)



In [ ]:
import matplotlib.pyplot as plt
accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs, accuracy, "bo", label="Training accuracy")
plt.plot(epochs, val_accuracy, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()

**evaluating the the new model**

In [ ]:
test_model = keras.models.load_model(
    "Flower_identification_with_augmentation.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
dropout_model= "Test accuracy: "+str(test_acc)
print(f"Test accuracy: {test_acc:.3f}")


In [ ]:
# from tensorflow.keras.preprocessing.image import ImageDataGenerator

# # Data augmentation
# datagen = ImageDataGenerator(
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     fill_mode='nearest'
# )

# # Model architecture
# model = keras.Sequential([
#     keras.Input(shape=(180, 180, 3)),
#     layers.Rescaling(1./255),
#     layers.Conv2D(32, (3, 3), activation='relu'),
#     layers.MaxPooling2D(2, 2),
#     layers.Conv2D(64, (3, 3), activation='relu'),
#     layers.MaxPooling2D(2, 2),
#     layers.Conv2D(128, (3, 3), activation='relu'),
#     layers.MaxPooling2D(2, 2),
#     layers.Flatten(),
#     layers.Dense(256, activation='relu'),
#     layers.Dropout(0.5),  # Dropout layer to reduce overfitting
#     layers.Dense(num_classes, activation='softmax')
# ])

# # Compile the model
# model.compile(optimizer='rmsprop',
#               loss='sparse_categorical_crossentropy',
#               metrics=['accuracy'])

# # Data augmentation flow for training dataset
# train_datagen = datagen.flow_from_directory(
#     'flowers_new_split/train',
#     target_size=(180, 180),
#     batch_size=32,
#     class_mode='sparse'
# )

# # Validation dataset
# validation_dataset = image_dataset_from_directory(
#     'flowers_new_split/validation',
#     image_size=(180, 180),
#     batch_size=32
# )

# # Train the model with data augmentation
# history = model.fit(
#     train_datagen,
#     epochs=30,
#     validation_data=validation_dataset,
#     callbacks=callbacks
# )


In [ ]:
# test_model = keras.models.load_model(
#     "Flower_identification_with_augmentation_dropout.keras")
# test_loss, test_acc = test_model.evaluate(test_dataset)
# aug_new_model= "Test accuracy: "+str(test_acc)
# print(f"Test accuracy: {test_acc:.3f}")


In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(180, 180, 3))
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(5, activation='softmax'))  # Adjust the output layer units according to your number of flower classes (5 in this case)
model.compile(optimizer=RMSprop(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])


history = model.fit(train_dataset, epochs=14, validation_data=validation_dataset, callbacks=callbacks)

In [ ]:
test_model = keras.models.load_model(
    "Flower_identification_with_augmentation_dropout.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
VGG16_new_model= "Test accuracy: "+str(test_acc)
print(f"Test accuracy: {test_acc:.3f}")


In [ ]:
model_result= [basic_model, augm_model, VGG16_model, dropout_model, VGG16_new_model]
for model in model_result:
  print(model)

In [ ]:
base_model